In [ ]:
#!/usr/bin/env python
# coding: utf-8

###################################### Librerias utilizadas ###################################

from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization, Input
#from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator     # Generador de imagenes.
from keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.applications.vgg19 import VGG19
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import cv2
import os

get_ipython().run_line_magic('matplotlib', 'inline')

# Librerias para metricas de desempeño y matriz de confusion
from sklearn import metrics                                  # Funcion para el calculo de metricas de desempeño, la de abajo hace lo mismo pero todo por separado.
from sklearn.metrics import confusion_matrix, f1_score, roc_curve, precision_score, recall_score, accuracy_score, roc_auc_score
from mlxtend.plotting import plot_confusion_matrix           # Paquete para ver matriz  de confusion en interfaz grafica.

import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

#Librerias para Generar data Augmentation
from keras.preprocessing.image import ImageDataGenerator 
from PIL import ImageFile

#Libreria para cargar modelo
from keras.models import load_model

# Libreria para obtener ruta de archivo actual
import pathlib

################################# Ingreso de parametros #########################################

pathInicial = str(pathlib.Path().absolute()) + "\\" # Path obtenido: ~/modelo

print("**** Ingreso de parametros ****\n")
modelVGG = int(input(" Modelo a utilizar: \n VGG16 --> Ingresar 0 \n VGG19 --> Ingresar 1\n"))
if not modelVGG:
    modelVGG = 16
    porcentaje = 0.99 # 8 errores en 100 fotos, 21 desconocidas (Bajando mas el valor, se comienzan a tener muchos errores)
                                                            #   (Aumentando el valor, los resultados son similares pero muchas imagenes desconocidos)
else:
    modelVGG = 19 
    porcentaje = 0.95 # 5 errores en 100 fotos, 13 desconocidas (Bajando  el valor  , se comienzan a tener muchos errores)
                                                            #   (Aumentando el valor, los resultados son similares pero muchas imagenes desconocidos)

tipoModelo = int(input("Tipo de set de datos: \n Diurno --> Ingresar 0 \n Nocturno --> Ingresar 1\n"))
if not tipoModelo:
    tipoModelo = "Diurno"
    pathAux = "Diurnas/"
    model_dir    = pathInicial + r"Backups\Diurno\model_VGG"+str(modelVGG)+"_500E_16B_"+str(tipoModelo)+".h5" 

else:
    tipoModelo = "Nocturno" 
    pathAux = "Nocturnas/"
    model_dir    = pathInicial + r"Backups\Nocturno\model_VGG"+str(modelVGG)+"_500E_16B_"+str(tipoModelo)+".h5" 

################################# Configuracion de parametros ###################################

width_shape  = 224  # Ancho de imagen (Ancho tipico de VGG16 y VGG19)
height_shape = 224  # Alto  de imagen (Alto tipico de VGG16 y VGG19)

############################### Excepciones ####################################################

if not os.path.exists(model_dir):
    raise FileNotFoundError("El archivo no existe o no se pudo abrir")
    
########################################### Predicción ######################################################################

custom_Model= load_model(model_dir)

names = ['Carpincho','Ciervo','Gato montes','Oveja','Puma','Vaca']     # Clases de nuestro dataset.
modelt= custom_Model                                                   # Modelo entrenado.

#administracion de directorios

#Renombramos imagenes
newImagesPath = pathInicial + r"newImages\\"
procesadasPath = pathInicial + r"Procesadas\\" + pathAux
lista = os.listdir(newImagesPath)                        # leemos lista de archivos. Se guardan nombre de carpeta o archivo.extension.

aux = 0
for file in lista:
    file = file[:-4]                                     # borramos ultimos 4 valores ".jpg"
    lista[aux]=file
    aux += 1

for g in range(len(lista)):                              # len(lista) devuelve cantidad de elementos
    path=newImagesPath + lista[g] + ".jpg"
    imaget = cv2.imread(path)
    imaget = cv2.resize((imaget), (width_shape, height_shape), interpolation = cv2.INTER_AREA) # Redimensionado de imagen
      
    #cv2.INTER_AREA: Esto se usa cuando necesitamos reducir una imagen.
    #cv2.INTER_CUBIC: Esto es lento pero más eficiente.
    #cv2.INTER_LINEAR: Esto se usa principalmente cuando se requiere hacer zoom. Esta es la técnica de interpolación predeterminada en OpenCV.
    xt     = np.asarray(imaget)
    xt     = preprocess_input(xt)                                       # Aplicado de preprocesamiento aplicado a las imagenes de entrenamiento y validacion.
    xt     = np.expand_dims(xt,axis=0)
    preds  = modelt.predict(xt)                                         # Predict devuelve las predicciones, el argumento con mayor valor de las predicciones correspondera con la clase mas probable
    
    print("prediccion N°:", g+1)
    print("animal detectado:", names[np.argmax(preds)])
    print("np.max(preds):", np.max(preds))
    print("preds:", preds)
    
    if (np.max(preds) >= porcentaje): # Si el porcentaje de certeza es < al valor asignado por el modelo almacenamos imagen en "desconocido"
        
        # Organizacion de directorios
        if (names[np.argmax(preds)]   == 'Carpincho'):
            directorioPath            =  procesadasPath + "Carpincho/"
            tipo                      =  "Carpincho"
    
        elif (names[np.argmax(preds)] == 'Ciervo'):
            directorioPath            =  procesadasPath + "Ciervo/"
            tipo                      =  "Ciervo"
            
        elif (names[np.argmax(preds)] == 'Gato montes'):
            directorioPath            =  procesadasPath + "Gato montes/"
            tipo                      =  "Gato montes"
                
        elif (names[np.argmax(preds)] == 'Oveja'):
            directorioPath            =  procesadasPath + "Oveja/"
            tipo                      =  "Oveja"
                
        elif (names[np.argmax(preds)] == 'Puma'):
            directorioPath            =  procesadasPath + "Puma/"
            tipo                      =  "Puma"
                
        elif (names[np.argmax(preds)] == 'Vaca'):
            directorioPath            =  procesadasPath + "Vaca/"
            tipo                      =  "Vaca"

    else:
        directorioPath                =  procesadasPath + "Desconocido/"
        tipo                          =  "Desconocido"
        print("porcentaje de certeza menor al " + str(porcentaje*100)+"% --> Objeto desconocido")
    
    print("\n")
    
                    
    os.rename(newImagesPath + lista[g] + ".jpg", directorioPath + lista[g] + "-" + tipo + ".jpg")
    g=g+1